 HW3: For Titanic dataset, do Pyspark to get the same result as we got with Pandas. Ignore the visualization parts. Due date for HW3 is September 30.

# Titanic Dataset Manipulation with PySpark

In [16]:
# Import PySpark goodies

from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
# Create a Spark Session

spark = SparkSession \
    .builder \
    .appName("Spark EDA of Titanic dataset") \
    .getOrCreate()

df = spark.read.csv('titanic.csv', header = 'True', inferSchema='True')
display(df)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [9]:
# Let's take a look at the first 5 rows data

df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [12]:
# Describe the data

df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## Passenger Exploration

In [15]:
# Number of null age values

total_rows = 891
age_values = 714
null_values = total_rows - age_values
print("Null values: ", null_values)

Null values:  177


In [17]:
# Survivors

df.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [18]:
# What sex survived?

df.groupBy("Sex", "Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [25]:
# Which P classes survived?

df.groupBy("Pclass","Survived").count().show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       0|   80|
|     3|       1|  119|
|     1|       1|  136|
|     2|       1|   87|
|     2|       0|   97|
|     3|       0|  372|
+------+--------+-----+



Most passengers were in P class 3, yet P class 1 had the most survivors despite having the fewest number of people.

### P class 1 had the most survivors and were likely prioritized

In [27]:
# Average age onboard

mean_age = df.select(mean('Age')).collect()[0][0]
print(mean_age)

29.69911764705882


In [29]:
# Embarked has only 2 missing values

df.groupBy("Embarked").count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



## Does survival rate have anything to do with family size of passengers?
### Creating a new feature called "Family" and "Alone" using Parch(parents/children) and SibSp(siblings/spouses). 

In [30]:
df = df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [31]:
df.groupBy("Family_Size").count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  161|
|          6|   12|
|          3|   29|
|          5|   22|
|          4|   15|
|          7|    6|
|         10|    7|
|          2|  102|
|          0|  537|
+-----------+-----+



In [32]:
df = df.withColumn('Alone',lit(0))

In [33]:
df = df.withColumn("Alone",when(df["Family_Size"] == 0, 1).otherwise(df["Alone"]))

In [34]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'Family_Size',
 'Alone']

In [37]:
# Label Encode columns with strings into numbers

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in ["Sex","Embarked"]]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

In [40]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+---------+--------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Family_Size|Alone|Sex_index|Embarked_index|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+---------+--------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|          1|    0|      0.0|           0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|          1|    0|      1.0|           1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|          0|    1|      1.0|           0.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|fe

In [47]:
# Let's get rid of columns we don't need

df = df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")
df.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|
|       0|     3|null|    0|    0| 8.4583|          0|    1|      0.0|           2.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|
|       0|     3| 2.0|    3|    1| 21.075|          4|    0|      0.0|           0.0|
|       1|     3|27.0|    0|    2|11.1333|          2|

# Modeling

### Train / Test

put feature into vector

In [48]:
feature = VectorAssembler(inputCols=df.columns[1:],outputCol="features")
feature_vector= feature.transform(df)

In [49]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2],seed = 11)

#### Need to clean the data a bit further, the "fit" method is not working
Tutorial: https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/5722190290795989/3865595167034368/8175309257345795/latest.html